In [ ]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
from scipy.ndimage import uniform_filter
from scipy.interpolate import RectBivariateSpline
from remeshing import get_area
from scipy.spatial import cKDTree

import os
from datetime import datetime, timedelta
import glob
from scipy.ndimage import gaussian_filter
from tqdm.notebook import tqdm

from utils import get_mesh_files, fill_gaps

%matplotlib inline

In [ ]:
def load_sid_data(ifile, mesh_src_file):
    with np.load(ifile) as d:
        u = d['u']
        v = -d['v']
        sid_unc = d['sid_unc']
    u[np.isnan(u)] = 0
    v[np.isnan(v)] = 0
    with np.load(mesh_src_file) as d:
        x0 = d['x']
        y0 = d['y']
        t0 = d['t']
    return u, v, sid_unc, x0, y0, t0

In [ ]:
# filter size (Gaissian filter was 5, but weights are smaller for larger distances)
size = 3

# should be 2210 to cover 6 years
n_steps = 2210 

In [ ]:
sia_dir = 'data2/Anton/sia/cdr_1991_2023'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
sid_dir = 'data2/Anton/sia_sid_cdr_postproc'
ifiles = sorted(glob.glob(f'{sid_dir}/*/*npz'))
idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
mask = np.load(mesh_init_file)['mask']
mask_f = gaussian_filter(mask.astype(float), 1, truncate=1)

# Accumulate and save SID uncertainty
$\sigma_{i} = \sqrt{<\sigma_{i-1}^2> + \sigma_S^2}$

$\sigma_S$ - uncertainty of smoothed SID from CDR

$\sigma_{i-1}$ - uncertainty from previous step

$<>$ - advection operation

$\sigma_S = \frac{\sqrt{\sum_j^N{\sigma_{Oj}}}}{\sqrt{N}}$

$\sigma_{Oj}$ - uncertaimty in neighbour $j$

$N$ - number of neighbours

In [ ]:
start_indices = [i for i,j in enumerate(idates) if j.year > 2018 and j.month == 9 and j.day == 5]
start_indices

In [ ]:
for start_idx in start_indices:
    stop_idx = min(start_idx + n_steps, len(idates))
    unc_sid_sum = None
    x = None
    start_date = idates[start_idx]
    odir = f'{unc_dir}/{start_date.year}'
    os.makedirs(odir, exist_ok=True)
    for i in tqdm(range(start_idx, stop_idx), total=n_steps):
        ifile = ifiles[i]
        idate = idates[i]
        ofile = f'{odir}/unc_sid_{start_date.strftime("%Y%m%d")}_{idate.strftime("%Y%m%d")}.npz'
        if os.path.exists(ofile):
            unc_sid_sum = np.load(ofile)['unc_sid']
            continue

        mesh_file, mesh_dst_file = get_mesh_files(idate, mesh_dir, mesh_init_file)
        u_grd, v_grd, unc_sid_grd, x, y, t = load_sid_data(ifile, mesh_file)
        unc_sid_fil = fill_gaps(unc_sid_grd, np.isnan(unc_sid_grd), distance=144)

        # compute uncertainty of drift field after smoothing
        unc_sid_smt = uniform_filter(unc_sid_fil**2, size=size) / size
        unc_sid_smt[u_grd == 0] = 0

        # interpolate uncertainty to mesh
        unc_sid = RectBivariateSpline(xc[1:-1:3], yc[1:-1:3], unc_sid_smt[::-1], kx=1, ky=1)(y[t].mean(axis=1), x[t].mean(axis=1), grid=False)

        if unc_sid_sum is None:
            # first time step
            unc_sid_sum = np.array(unc_sid)
        else:
            # advect unc_sid_sum (from previous time step)
            src2dst = np.load(mesh_file)['src2dst']
            weights = np.load(mesh_file)['weights']
            unc_sid_sum_pro = np.zeros(src2dst[:,1].max()+1)
            np.add.at(unc_sid_sum_pro, src2dst[:,1], unc_sid_sum[src2dst[:,0]] * weights)
            # compute uncertainty of drift field after advection
            unc_sid_sum = np.hypot(unc_sid, unc_sid_sum_pro)
        np.savez(ofile, unc_sid=unc_sid_sum)

    print(start_date)
    fig, axs = plt.subplots(1, 1, figsize=(6, 6))
    trp0 = axs.tripcolor(x, y, t, unc_sid_sum, cmap='jet')
    plt.colorbar(trp0, ax=axs)
    plt.show()

In [ ]:
stop_idx

In [ ]:
unc_sid_sum = None
unc_sid_sum_all = {}

unc_sic_sum = None
unc_sic_sum_all = {}

sic_min = None

unc_sic_sid_all = {}

n_steps = 120

# 2020
#start = 10840
#stop = start + n_steps

# 1991
start = 247
stop = start + n_steps


for i in tqdm(range(start, stop), total=n_steps):
    ifile = ifiles[i]
    idate = idates[i]
    #print(ifile, idate)

    mesh_file, mesh_dst_file = get_mesh_files(idate, mesh_dir, mesh_init_file)
    u0, v0, tri0 = load_data(ifile, mesh_file)

    # load uncertainty of drift field
    sid_file = f'/Data/sim/data/OSISAF_ice_drift_CDR_v1p0_merged/{idate.strftime("%Y")}/{idate.strftime("%m")}/ice_drift_nh_ease2-750_cdr-v1p0_24h-{idate.strftime("%Y%m%d")}1200.nc'
    with Dataset(sid_file) as dds:
        uncrt = dds['uncert_dX_and_dY'][0].filled(np.nan)
    uncrt_filled = fill_gaps(uncrt, np.isnan(uncrt), distance=144)

    # compute uncertainty of drift field after smoothing
    uncrt_mean = uniform_filter(uncrt_filled**2, size=size) / size
    uncrt_mean[u0 == 0] = 0

    # interpolate uncertainty to mesh
    x0, y0, t0 = tri0.x, tri0.y, tri0.triangles
    unc_sid0 = RectBivariateSpline(xc[1:-1:3], yc[1:-1:3], uncrt_mean[::-1], kx=1, ky=1)(y0[t0].mean(axis=1), x0[t0].mean(axis=1), grid=False)

    if unc_sid_sum is None:
        # first time step
        unc_sid_sum = np.array(unc_sid0)
    else:
        # advect unc_sid_sum (from previous time step)
        src2dst = np.load(mesh_file)['src2dst']
        weights = np.load(mesh_file)['weights']
        unc_sid_sum_pro = np.zeros(src2dst[:,1].max()+1)
        np.add.at(unc_sid_sum_pro, src2dst[:,1], unc_sid_sum[src2dst[:,0]] * weights)
        # compute uncertainty of drift field after advection
        unc_sid_sum = np.hypot(unc_sid0, unc_sid_sum_pro)

    unc_sid_sum_all[idate] = unc_sid_sum

    # load uncertainty of sea ice concentration
    sic_file_mask = f'/Data/sim/data/OSISAF_ice_conc_CDR_v3p0/{idate.strftime("%Y")}/{idate.strftime("%m")}/ice_conc_nh_ease2-*_{idate.strftime("%Y%m%d")}1200.nc'
    sic_file = glob.glob(sic_file_mask)[0]
    with Dataset(sic_file) as dds:
        uncrt_sic = dds['total_standard_uncertainty'][0].filled(np.nan)
    uncrt_sic_filled = fill_gaps(uncrt_sic, np.isnan(uncrt_sic), distance=144)
    
    # interpolate SIC uncertainty to mesh
    unc_sic0 = RectBivariateSpline(xc, yc, uncrt_sic_filled[::-1], kx=1, ky=1)(y0[t0].mean(axis=1), x0[t0].mean(axis=1), grid=False)

    sic_file = mesh_file.replace('mesh', 'sic')
    c0 = np.load(sic_file)['c']

    if unc_sic_sum is None:
        # first time step
        unc_sic_sum = np.array(unc_sic0)
        sic_min = np.array(c0)
    else:
        # advect unc_sic_sum (from previous time step)
        unc_sic_sum_pro = np.zeros(src2dst[:,1].max()+1)
        np.add.at(unc_sic_sum_pro, src2dst[:,1], unc_sic_sum[src2dst[:,0]] * weights)
        
        # advect sic_min from previous time step
        sic_min_pro = np.zeros(src2dst[:,1].max()+1)
        np.add.at(sic_min_pro, src2dst[:,1], sic_min[src2dst[:,0]] * weights)

        # keep uncertainty of the minimal value of SIC
        min_sic_ids = c0 < sic_min_pro
        unc_sic_sum = unc_sic_sum_pro
        unc_sic_sum[min_sic_ids] = unc_sic0[min_sic_ids]
        
        # keep min ice concentration
        sic_min = np.min([sic_min_pro, c0], axis=0)
        # sum up uncertainties of SIC field
        #unc_sic_sum = np.hypot(unc_sic0, unc_sic_sum_pro)

    # compute uncertainty of SIC field as STD in the area defined by the uncert of drift field
    el_size = get_area(x0, y0, t0)**0.5
    sid_unc_rel_to_el_size = np.abs(unc_sid_sum - el_size) / el_size

    unc_sic_sid = np.zeros_like(unc_sic_sum)
    el_size_mean = el_size.mean()

    min_rel_size_factor = 2
    min_rel_sizes = [2, 4, 6, 8, 10, 12]

    for min_rel_size in min_rel_sizes:
        min_abs_size = el_size_mean * min_rel_size * min_rel_size_factor

        x0el = x0[t0].mean(axis=1)
        y0el = y0[t0].mean(axis=1)

        uncert_elems = np.nonzero(
            (sid_unc_rel_to_el_size >= min_rel_size) * 
            (sid_unc_rel_to_el_size < min_rel_size+min_rel_size_factor)
            )[0]
        if len(uncert_elems) == 0:
            continue
        
        x_unc_sid = x0el[uncert_elems]
        y_unc_sid = y0el[uncert_elems]
        tree_unc = cKDTree(np.c_[x_unc_sid, y_unc_sid])
        tree_all = cKDTree(np.c_[x0el, y0el])
        indices = tree_unc.query_ball_tree(tree_all, min_abs_size)

        for i, indx in enumerate(indices):
            if len(indx) > 0:
                unc_sic_sid[uncert_elems[i]] = c0[indx].std()
    
    unc_sic_sum_all[idate] = unc_sic_sum
    unc_sic_sid_all[idate] = unc_sic_sid

    #fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    #axs[0].tripcolor(x0, y0, t0, unc_sid_sum, cmap='jet', clim=[0, 30])
    #axs[0].set_title('Drift uncertainty')
    #axs[1].tripcolor(x0, y0, t0, unc_sic_sum, cmap='jet', clim=[0, 50])
    #axs[1].set_title('SIC uncertainty')
    #plt.show()


In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(24, 6))
trp0 = axs[0].tripcolor(x0, y0, t0, unc_sid_sum, cmap='jet')
plt.colorbar(trp0, ax=axs[0])
trp1 = axs[1].tripcolor(x0, y0, t0, unc_sic_sum, cmap='jet')
plt.colorbar(trp1, ax=axs[1])
trp2 = axs[2].tripcolor(x0, y0, t0, unc_sic_sid, cmap='jet')
plt.colorbar(trp2, ax=axs[2])
trp3 = axs[3].tripcolor(x0, y0, t0, 
                        np.hypot(
                            unc_sic_sum,
                            unc_sic_sid), cmap='jet')
plt.colorbar(trp3, ax=axs[3])
plt.show()
